In [1]:
import os
import cv2
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from tensorflow.keras.preprocessing import image
from tensorflow import keras
from tensorflow.keras.applications import resnet50, ResNet50
from tensorflow.keras.preprocessing import image


In [11]:
model = tf.keras.models.load_model(r'E:\AImodel\models\Face-ResNet-Plank-remove-model')


In [12]:
def preprocess_image(image_path, target_size=(128, 128)):
    """이미지 파일을 불러와 전처리하는 함수"""
    img = image.load_img(image_path, target_size=target_size)
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # 모델 예측을 위해 차원 확장
    img_array /= 255.0  # 이미지를 0과 1 사이로 스케일링
    return img_array

def load_and_preprocess_from_directory(directory_path, target_size=(128, 128)):
    """지정된 디렉토리 내의 모든 이미지를 불러와 전처리하는 함수"""
    processed_images = []
    for file_name in os.listdir(directory_path):
        file_path = os.path.join(directory_path, file_name)
        if file_path.lower().endswith(('.png', '.jpg', '.jpeg')):
            img = preprocess_image(file_path, target_size=target_size)
            processed_images.append(img)
    return np.vstack(processed_images)  # 전처리된 이미지들을 하나의 numpy 배열로 합침

dir = r'E:\AItemp\testdata\plank test'
preprocessed_images = load_and_preprocess_from_directory(dir)
predictions = model.predict(preprocessed_images)


1/1 [==============================] - 1s 727ms/step


In [13]:
# 모델의 출력층에 접근하여 클래스의 개수 얻기
output_layer = model.layers[-1]  # 모델의 마지막 층 (출력층)을 가져옴
num_classes = output_layer.units  # 출력층의 유닛 수를 가져옴, 이는 클래스의 수와 동일

print(f'모델은 총 {num_classes}개의 클래스를 가지고 있습니다.')
class_labels = []
# 클래스 라벨 정의 (예: num_classes 개의 클래스)
for i in range(num_classes):
    class_labels.append(f'class{i+1}')

# 예를 들어, num_classes가 3이라면 class_labels는 ['class1', 'class2', 'class3']이 됩니다.


# predictions 배열에서 각 입력에 대한 모든 클래스의 확률 출력
for i, prediction in enumerate(predictions):
    print(f'Image {i+1}:')
    for j, class_label in enumerate(class_labels):
        print(f'  {class_label}: {prediction[j]:.2f}%')
    print()  # 각 이미지의 예측 결과 사이에 공백 추가


모델은 총 4개의 클래스를 가지고 있습니다.
Image 1:
  class1: 0.00%
  class2: 1.00%
  class3: 0.00%
  class4: 0.00%

Image 2:
  class1: 0.00%
  class2: 1.00%
  class3: 0.00%
  class4: 0.00%

Image 3:
  class1: 0.00%
  class2: 1.00%
  class3: 0.00%
  class4: 0.00%

Image 4:
  class1: 0.00%
  class2: 1.00%
  class3: 0.00%
  class4: 0.00%

Image 5:
  class1: 0.00%
  class2: 1.00%
  class3: 0.00%
  class4: 0.00%

Image 6:
  class1: 0.00%
  class2: 1.00%
  class3: 0.00%
  class4: 0.00%

Image 7:
  class1: 0.00%
  class2: 1.00%
  class3: 0.00%
  class4: 0.00%

Image 8:
  class1: 0.00%
  class2: 1.00%
  class3: 0.00%
  class4: 0.00%

Image 9:
  class1: 0.00%
  class2: 1.00%
  class3: 0.00%
  class4: 0.00%

Image 10:
  class1: 0.00%
  class2: 1.00%
  class3: 0.00%
  class4: 0.00%

Image 11:
  class1: 0.00%
  class2: 1.00%
  class3: 0.00%
  class4: 0.00%

Image 12:
  class1: 0.00%
  class2: 1.00%
  class3: 0.00%
  class4: 0.00%

Image 13:
  class1: 0.00%
  class2: 1.00%
  class3: 0.00%
  class4: 0.00%

Image 14:

In [14]:
# 예측 배열에서 각 입력에 대해 최고 확률을 가진 클래스의 인덱스를 찾음
predicted_classes = np.argmax(predictions, axis=1)

# 각 클래스별 이미지 수를 계산하기 위한 딕셔너리 초기화
class_counts = {class_label: 0 for class_label in class_labels}

# 예측된 클래스별로 이미지 수를 계산
for class_index in predicted_classes:
    class_label = class_labels[class_index]
    class_counts[class_label] += 1

# 결과 출력
for class_label, count in class_counts.items():
    print(f'{class_label}: {count}개의 이미지가 이 클래스가 가장 높은 확률을 가집니다.')


class1: 0개의 이미지가 이 클래스가 가장 높은 확률을 가집니다.
class2: 24개의 이미지가 이 클래스가 가장 높은 확률을 가집니다.
class3: 0개의 이미지가 이 클래스가 가장 높은 확률을 가집니다.
class4: 0개의 이미지가 이 클래스가 가장 높은 확률을 가집니다.


In [7]:
def make_gradcam_heatmap(img_array, model, last_conv_layer_name, pred_index=None):
    # Grad-CAM 알고리즘 구현
    grad_model = tf.keras.models.Model(
        [model.inputs], [model.get_layer(last_conv_layer_name).output, model.output]
    )

    with tf.GradientTape() as tape:
        last_conv_layer_output, preds = grad_model(img_array)
        if pred_index is None:
            pred_index = tf.argmax(preds[0])
        class_channel = preds[:, pred_index]

    # 이 클래스에 대한 최종 컨볼루션 레이어의 그래디언트
    grads = tape.gradient(class_channel, last_conv_layer_output)

    # 가중치 평균값
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

    # 최종 컨볼루션 레이어의 출력에 가중치를 곱함
    last_conv_layer_output = last_conv_layer_output[0]
    heatmap = last_conv_layer_output @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)

    # ReLU 활성화 및 정규화
    heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)
    return heatmap.numpy()


In [8]:
# 사전 학습된 신경망 모델을 불러오고 구조 확인
model = ResNet50(weights='imagenet')

# 폴더 내의 모든 jpg 파일에 대해 처리
image_folder = r'E:\AItemp\test\plank test'  # 대상 폴더 경로
image_files = [os.path.join(image_folder, f) for f in os.listdir(image_folder) if f.endswith('.jpg')]

for image_path in image_files:
    img = image.load_img(image_path, target_size=(224, 224))
    
    # 영상을 신경망 형태로 변환
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = resnet50.preprocess_input(x)

    last_conv_layer = model.get_layer("conv5_block3_out")
    model_1 = keras.Model(model.inputs, last_conv_layer.output)

    input_2 = keras.Input(shape=last_conv_layer.output.shape[1:])
    x_2 = model.get_layer("avg_pool")(input_2)
    x_2 = model.get_layer("predictions")(x_2)
    model_2 = keras.Model(input_2, x_2)

    with tf.GradientTape() as tape:
        output_1 = model_1(x)
        tape.watch(output_1)
        preds = model_2(output_1)
        class_id = tf.argmax(preds[0])
        output_2 = preds[:, class_id]

    grads = tape.gradient(output_2, output_1)
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

    output_1 = output_1.numpy()[0]
    pooled_grads = pooled_grads.numpy()
    for i in range(pooled_grads.shape[-1]):
        output_1[:, :, i] *= pooled_grads[i]
    heatmap = np.mean(output_1, axis=-1)

    heatmap = np.maximum(heatmap, 0) / np.max(heatmap)
    
    # 열지도를 입력 영상에 씌움
    img = image.load_img(image_path)
    img = image.img_to_array(img)
    heatmap = np.uint8(255 * heatmap)

    jet = cm.get_cmap("jet")
    color = jet(np.arange(256))[:, :3]
    color_heatmap = color[heatmap]

    color_heatmap = keras.preprocessing.image.array_to_img(color_heatmap)
    color_heatmap = color_heatmap.resize((img.shape[1], img.shape[0]))
    color_heatmap = keras.preprocessing.image.img_to_array(color_heatmap)

    overlay_img = color_heatmap * 0.4 + img
    overlay_img = keras.preprocessing.image.array_to_img(overlay_img)

    # 결과 이미지 디스플레이
    plt.figure()
    plt.imshow(overlay_img)
    plt.show()


FileNotFoundError: [WinError 3] 지정된 경로를 찾을 수 없습니다: 'E:\\AItemp\\test\\plank test'

In [ ]:
# import numpy as np
# import tensorflow as tf
# from tensorflow import keras
# from tensorflow.keras.applications import resnet50, ResNet50
# from tensorflow.keras.preprocessing import image
# import matplotlib.pyplot as plt
# import matplotlib.cm as cm
 
# #사전 학습된 신경망 모델을 불러오고 구조 확인
# model = ResNet50(weights='imagenet')
# #model.summary()
 
# #지정된 영상을 불러와 크기 조정하고 화면에 디스플레이
# image_path = r'E:\AItemp\test\plank test'#r'E:\AItemp\BicycleCrunch\training\BicycleCrunch_505\505-Z1_C-0000016.jpg'
# img = image.load_img(image_path, target_size = (224,224))
# plt.matshow(img)
 
# #영상을 신경망 형태로 변환
# x=image.img_to_array(img)
# x=np.expand_dims(x,axis=0)
# x=resnet50.preprocess_input(x)
 
# #신경망 모델의 특 징 추출 부분에서 마지막 층을 지정
# #특징 추출 부분만으로 구성된 model_1만들기
# last_conv_layer = model.get_layer("conv5_block3_out")
 
# model_1= keras.Model(model.inputs, last_conv_layer.output)
 
# #분류 (전역평균풀링 또는 완전연결층) 부분만으로 구성된 model__2만들기
# input_2 = keras.Input(shape=last_conv_layer.output.shape[1:])
# x_2 = model.get_layer("avg_pool")(input_2)
# x_2 = model.get_layer("predictions")(x_2)
# model_2=keras.Model(input_2,x_2)
 
# #GradientTape함수를 이용한 그레디언트 계산
# with tf.GradientTape() as tape:
#     output_1 = model_1(x)
#     tape.watch(output_1) #마지막 층으로 미분하기 위한 준비
#     preds = model_2(output_1)
#     class_id = tf.argmax(preds[0])
#     output_2 = preds[:,class_id]
 
# grads = tape.gradient(output_2, output_1) #그레디언트 계산
# pooled_grads = tf.reduce_mean(grads,axis=(0,1,2)) #식5 적용
 
# output_1 = output_1.numpy()[0]
# pooled_grads = pooled_grads.numpy()
# for i in range(pooled_grads.shape[-1]):
#     output_1[:,:,i]*=pooled_grads[i]
# heatmap=np.mean(output_1, axis=-1)
 
# heatmap =np.maximum(heatmap, 0)/np.max(heatmap) #정규화
# plt.matshow(heatmap)
 
# #열지도를 입력 영상에 씌움
# img =image.load_img(image_path) #입력 영상을 다시 받음
 
# img=image.img_to_array(img)
# heatmap=np.uint8(255*heatmap) # [0,255]로 변환
 
# jet = cm.get_cmap("jet") #jet 컬러맵으로 표시
# color = jet(np.arange(256))[:,:3]
# color_heatmap = color[heatmap]
 
# color_heatmap = keras.preprocessing.image.array_to_img(color_heatmap)
# color_heatmap = color_heatmap.resize((img.shape[1], img.shape[0]))
# color_heatmap = keras.preprocessing.image.img_to_array(color_heatmap)
 
# overlay_img= color_heatmap*0.4+img #덧씌움
# overlay_img = keras.preprocessing.image.array_to_img(overlay_img)
# plt.matshow(overlay_img)